In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 载入数据
mnist = input_data.read_data_sets('MNIST_DATA',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

# 初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)  # 生成一个截断的正态分布
    return tf.Variable(initial)

# 初始化偏置值:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x, W):
    # x:输入数据,tensor 形状[batch, height, width, 通道]
    # W:卷积核/滤波器, 形状[height, width, 输入通道, 输出通道]
    # stride[0]=stride[3]=1, stride[1]代表x方向的步长, stride[3]代表y方向的步长
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

# 池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# 改变x的格式,转换成4D的向量[batch, in_heigjt, in_width, in_channel]
x_image = tf.reshape(x, [-1, 28, 28, 1])

## -------------------------------------------------- 第一次卷积池化操作
# 初始化第一个卷积层的全值和偏置
W_conv1 = weight_variable([5, 5, 1, 32])  # 5x5的采样窗口,32个卷积核
b_conv1 = bias_variable([32])  # 每一个卷积核对应一个偏置

# 把x_image和权值向量进行卷积,再加上偏置,然后用relu激活
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


## -------------------------------------------------- 第二次卷积池化操作
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

"""
通过上面的操作:
1: 一张28*28的图片,进行第一次卷积后(窗口大小1*1,自动填充边缘),图片的大小还是28*28*32(32通道).
2:进行第一次池化后(窗口大小2*2),图片变成14*14.

3:第二次卷积后,图片大小14*14
4:第二次池化后,图片大小7*7*64(64通道, 即64个卷积核进行处理,获得64个子图)
"""

## ------------------------------------------------- 第一个全连接层
# 初始化全连接层的 w 和 b
W_fc1 = weight_variable([7*7*64, 1024])  # 上一层有64个神经元,每个图7*7,全连接层有1024个神经元
b_fc1 = bias_variable([1024])

# 将第二个池化层的输出,扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
# 计算第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# ------------------------------------------------- 第=个全连接层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# 计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# 交叉熵代价函数
cross_loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_loss)
# 计算accuracy
cross_prediction = tf.equal(tf.math.argmax(
    prediction, 1), tf.math.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(cross_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={
                     x: batch_xs, y: batch_ys, keep_prob: 0.8})

        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob:1.0})
        print('iter '+str(epoch)+', accuracy: ' + str(acc))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
